In [5]:
from itertools import permutations, product

import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import RepeatedStratifiedKFold, train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, auc, confusion_matrix, balanced_accuracy_score, precision_recall_curve, auc, roc_curve, roc_auc_score
from sklearn.inspection import permutation_importance
from sklearn import preprocessing

from sklearn.neural_network import MLPClassifier

from imblearn.ensemble import BalancedBaggingClassifier
from xgboost import XGBClassifier

import numpy as np
from numpy import mean,std

import pickle

from ctgan import CTGANSynthesizer

from os import path

import matplotlib.pyplot as plt

from warnings import simplefilter
from collections import OrderedDict
from sklearn.ensemble import VotingClassifier

z = False
try:
    %cd ../utils
except:
    z=True
import model_utils
import validation_utils
if z:
    %cd ..


simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None

/Users/benjaminli/Documents/coding/scires/project/utils


In [6]:
%cd ..
os.getcwd()

/Users/benjaminli/Documents/coding/scires/project


'/Users/benjaminli/Documents/coding/scires/project'

In [7]:
df = pd.read_csv('data/info.csv')
X_train, X_test, y_train, y_test = train_test_split(df.loc[:, df.columns != 'isZoonotic'], df['isZoonotic'], test_size=0.6, random_state=None)

In [15]:
# https://gist.github.com/tomquisel/a421235422fdf6b51ec2ccc5e3dee1b4
class VotingClassifier(object):
    """Stripped-down version of VotingClassifier that uses prefit estimators"""
    def __init__(self, estimators, voting='hard', weights=None):
        self.estimators = [e[1] for e in estimators]
        self.named_estimators = dict(estimators)
        self.voting = voting
        self.weights = weights

    def fit(self, X, y, sample_weight=None):
        raise NotImplementedError
        
    def predict(self, X):
        """ Predict class labels for X.
        Parameters
        ----------
        X : {array-like, sparse matrix}, shape = [n_samples, n_features]
            Training vectors, where n_samples is the number of samples and
            n_features is the number of features.
        Returns
        ----------
        maj : array-like, shape = [n_samples]
            Predicted class labels.
        """

        # check_is_fitted(self, 'estimators')
        if self.voting == 'soft':
            maj = np.argmax(self.predict_proba(X), axis=1)

        else:  # 'hard' voting
            predictions = self._predict(X)
            maj = np.apply_along_axis(lambda x:
                                      np.argmax(np.bincount(x,
                                                weights=self.weights)),
                                      axis=1,
                                      arr=predictions.astype('int'))
        return maj

    def _collect_probas(self, X):
        """Collect results from clf.predict calls. """
        return np.asarray([clf.predict_proba(X) for clf in self.estimators])

    def _predict_proba(self, X):
        """Predict class probabilities for X in 'soft' voting """
        if self.voting == 'hard':
            raise AttributeError("predict_proba is not available when"
                                 " voting=%r" % self.voting)
        # check_is_fitted(self, 'estimators')
        avg = np.average(self._collect_probas(X), axis=0, weights=self.weights)
        return avg

    @property
    def predict_proba(self):
        """Compute probabilities of possible outcomes for samples in X.
        Parameters
        ----------
        X : {array-like, sparse matrix}, shape = [n_samples, n_features]
            Training vectors, where n_samples is the number of samples and
            n_features is the number of features.
        Returns
        ----------
        avg : array-like, shape = [n_samples, n_classes]
            Weighted average probability for each class per sample.
        """
        return self._predict_proba

    def transform(self, X):
        """Return class labels or probabilities for X for each estimator.
        Parameters
        ----------
        X : {array-like, sparse matrix}, shape = [n_samples, n_features]
            Training vectors, where n_samples is the number of samples and
            n_features is the number of features.
        Returns
        -------
        If `voting='soft'`:
          array-like = [n_classifiers, n_samples, n_classes]
            Class probabilities calculated by each classifier.
        If `voting='hard'`:
          array-like = [n_samples, n_classifiers]
            Class labels predicted by each classifier.
        """
        # check_is_fitted(self, 'estimators')
        if self.voting == 'soft':
            return self._collect_probas(X)
        else:
            return self._predict(X)

    def _predict(self, X):
        """Collect results from clf.predict calls. """
        return np.asarray([clf.predict(X) for clf in self.estimators]).T

In [23]:
origGradBoost = pickle.load(open('models/curr_models/gradBoost.pkl', 'rb'))
origXGBoost = pickle.load(open('models/curr_models/xgBoost.pkl', 'rb'))
origLogReg = pickle.load(open('models/curr_models/lrmodel.pkl', 'rb'))
origKNN = pickle.load(open('models/curr_models/knn.pkl', 'rb'))
origRandForest = pickle.load(open('models/curr_models/randforest.pkl', 'rb'))
origMLP = pickle.load(open('models/curr_models/mlpClassifier.pkl', 'rb'))

nardusGradBoost = pickle.load(open('models/nardus_gridsearch.pkl', 'rb'))

estimators=[('gradBoost', origGradBoost), ('nardusgradBoost', nardusGradBoost), ('rf', origRandForest), ('mlp', origMLP)]

ensemble = VotingClassifier(estimators, voting='hard')

print(accuracy_score(y_test, ensemble.predict(X_test)))

print(accuracy_score(y_test, origGradBoost.predict(X_test)))
print(accuracy_score(y_test, origKNN.predict(X_test)))
print(accuracy_score(y_test, origMLP.predict(X_test)))
print(accuracy_score(y_test, nardusGradBoost.predict(X_test)))


0.9849770236832803
0.8506539413220219
0.9211735595616826
0.9814422057264051
0.9906327324142806
